# NUEVOS DATOS DE NIGERIA

link al dataset: https://zenodo.org/record/1252141

In [1]:
import pandas as pd
import glob
import ssqueezepy #para convertir la señal en imagen
from ssqueezepy import cwt
from ssqueezepy.visuals import plot, imshow
import os
import mne #eeg analysis library
import scipy.io
#import torch.nn as nn
#import torch
import numpy as np
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
metadata = pd.read_csv('C:\\Users\\47575909\\Desktop\\metadata_nigeria.csv')
metadata

,subject.id,recordedPeriod,startTime,session.id,first_condition,remarks,Group,csv.file
0,6,270,26/9/2016 13:13,1,open,NaN,control,signal-6-1.csv.gz
1,9,271,26/9/2016 13:30,1,closed,NaN,control,signal-9-1.csv.gz
2,10,272,26/9/2016 13:36,1,open,eyes closed at 2:40,control,signal-10-1.csv.gz
3,11,274,26/9/2016 13:42,2,closed,no.11.1 failed >> 11.2 is the right one,control,signal-11-2.csv.gz
4,11,1,26/9/2016 13:42,1,closed,no.11.1 failed >> 11.2 is the right one,control,signal-11-1.csv.gz
...,...,...,...,...,...,...,...,...
220,634,268,1/10/2016 16:25,1,closed,NaN,epilepsy,signal-634-1.csv.gz
221,635,268,1/10/2016 16:31,1,open,"Every day a convulsion + snapt het niet goed, ...",epilepsy,signal-635-1.csv.gz
222,636,268,1/10/2016 16:36,1,closed,NaN,epilepsy,signal-636-1.csv.gz
223,637,268,1/10/2016 16:43,1,open,NaN,epilepsy,signal-637-1.csv.gz


In [3]:
#veo la mierda de los ids
groups = metadata.groupby('subject.id')

grp_keys = list(groups.groups.keys())
print(len(grp_keys))

222


In [4]:
metadata = metadata.drop(4)

metadata.reset_index(drop=True, inplace=True)

metadata

,subject.id,recordedPeriod,startTime,session.id,first_condition,remarks,Group,csv.file
0,6,270,26/9/2016 13:13,1,open,NaN,control,signal-6-1.csv.gz
1,9,271,26/9/2016 13:30,1,closed,NaN,control,signal-9-1.csv.gz
2,10,272,26/9/2016 13:36,1,open,eyes closed at 2:40,control,signal-10-1.csv.gz
3,11,274,26/9/2016 13:42,2,closed,no.11.1 failed >> 11.2 is the right one,control,signal-11-2.csv.gz
4,12,271,26/9/2016 13:47,1,open,NaN,control,signal-12-1.csv.gz
...,...,...,...,...,...,...,...,...
219,634,268,1/10/2016 16:25,1,closed,NaN,epilepsy,signal-634-1.csv.gz
220,635,268,1/10/2016 16:31,1,open,"Every day a convulsion + snapt het niet goed, ...",epilepsy,signal-635-1.csv.gz
221,636,268,1/10/2016 16:36,1,closed,NaN,epilepsy,signal-636-1.csv.gz
222,637,268,1/10/2016 16:43,1,open,NaN,epilepsy,signal-637-1.csv.gz


In [5]:
metadata = metadata.drop('startTime', axis=1)
metadata = metadata.drop('first_condition', axis=1)
metadata = metadata.drop('recordedPeriod', axis=1)
metadata = metadata.drop('remarks', axis=1)
metadata = metadata.drop('session.id', axis=1)

metadata['csv.file'] = metadata['csv.file'].str.replace('.csv.gz', '.csv')

metadata

,subject.id,Group,csv.file
0,6,control,signal-6-1.csv
1,9,control,signal-9-1.csv
2,10,control,signal-10-1.csv
3,11,control,signal-11-2.csv
4,12,control,signal-12-1.csv
...,...,...,...
219,634,epilepsy,signal-634-1.csv
220,635,epilepsy,signal-635-1.csv
221,636,epilepsy,signal-636-1.csv
222,637,epilepsy,signal-637-1.csv


In [19]:
noseque = pd.read_csv('C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-10-1.csv')
dataframe = noseque.iloc[:, 1:15].copy()
dataframe

,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8
0,4004,-227.315789,4045,106.126316,124.989474,3705,321.400000,3866,4093,-146.810526,-55.010526,3976,3909,-180.136842
1,3998,-230.400000,4043,94.842105,121.400000,3698,316.273684,3862,4092,-148.863158,-59.115789,3972,3908,-190.915789
2,3997,-239.631579,4045,92.273684,120.884211,3696,314.221053,3861,4081,-147.315789,-58.600000,3972,3908,-192.452632
3,4005,-243.221053,4046,101.505263,125.494737,3701,320.368421,3863,4142,-147.831579,-54.505263,3974,3910,-186.294737
4,4003,-241.168421,4044,100.484211,127.547368,3699,323.957895,3860,4244,-152.452632,-50.915789,3973,3912,-190.915789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34811,3902,82.536842,4106,199.452632,90.115789,3622,-338.715789,3896,4326,54.221053,-49.884211,4043,3741,-251.936842
34812,3894,75.863158,4104,200.989474,88.578947,3618,-338.715789,3896,4326,44.473684,-52.452632,4036,3742,-260.136842
34813,3903,77.400000,4099,202.536842,91.652632,3632,-339.221053,3909,4323,42.936842,-60.652632,4034,3738,-259.631579
34814,3913,86.631579,4104,206.631579,92.168421,3644,-341.789474,3914,4321,43.957895,-66.810526,4035,3732,-256.042105


In [21]:
dataset = pd.DataFrame()

for archivo_csv in os.listdir('C:\\Users\\47575909\\Desktop\\EEGs_Nigeria'):
    if archivo_csv.endswith('.csv'):
        archivo_csv = archivo_csv.replace(' ', '')
        
        # Obtener el nombre de archivo sin la extensión .csv
        archivo_base = archivo_csv.replace('.csv', '')

        # Verifica si hay filas en metadata que cumplen con la condición de búsqueda
        matching_rows = metadata.loc[metadata['csv.file'] == archivo_base]

        if not matching_rows.empty:  # Verifica si se encontraron coincidencias
            subject_id, label = matching_rows[['subject.id', 'Group']].values[0]

            # Leer el archivo CSV de EEG
            eeg_data = pd.read_csv(os.path.join(directorio_eeg, archivo_csv))

            # Agregar las columnas de subject ID y etiqueta al DataFrame de EEG
            eeg_data['subject.id'] = subject_id
            eeg_data['Group'] = label

            # Agregar los datos del EEG al conjunto de datos combinado
            dataset = pd.concat([dataset, eeg_data])

            print(f"Se encontró una coincidencia para {archivo_csv}: subject_id={subject_id}, Group={label}")
        else:
            print(f"No se encontraron coincidencias para {archivo_csv}")

# Puedes guardar el conjunto de datos combinado en un nuevo archivo CSV si lo deseas
dataset.to_csv('new_eeg_data.csv', index=False)

No se encontraron coincidencias para signal-10-1.csv
No se encontraron coincidencias para signal-100-1.csv
No se encontraron coincidencias para signal-101-1.csv
No se encontraron coincidencias para signal-102-1.csv
No se encontraron coincidencias para signal-103-1.csv
No se encontraron coincidencias para signal-104-1.csv
No se encontraron coincidencias para signal-105-1.csv
No se encontraron coincidencias para signal-106-1.csv
No se encontraron coincidencias para signal-107-1.csv
No se encontraron coincidencias para signal-108-1.csv
No se encontraron coincidencias para signal-109-1.csv
No se encontraron coincidencias para signal-11-2.csv
No se encontraron coincidencias para signal-110-1.csv
No se encontraron coincidencias para signal-112-1.csv
No se encontraron coincidencias para signal-113-1.csv
No se encontraron coincidencias para signal-114-1.csv
No se encontraron coincidencias para signal-116-1.csv
No se encontraron coincidencias para signal-12-1.csv
No se encontraron coincidencias

In [10]:
dataset

""


In [ ]:
#CHEEEEEE CHONA DICE QUE PRIMERO CONVIERTAS CADA UNO POR SEPARADO AUNQUE CREO QUE LA VERDADD ES QUE ES LO MISMO PERO BUENO, GRACIAS CHONA :))